In [1]:
from gradio_client import Client, handle_file  # Changed from 'file' to 'handle_file'
from pathlib import Path
import time
import json
import shutil
import os

In [2]:
def load_config(config_file='config.json'):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

config = load_config()

In [3]:
# Initialize client
client = Client("Yifeng-Liu/satellite-image-roofs-auto-annotation")

Loaded as API: https://yifeng-liu-satellite-image-roofs-auto-annotation.hf.space ✔


In [4]:
def process_single_image(image_path, output_dir="rt_detr_outputs"):
    # Setup output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print("Checking image path...")
    image_exists = Path(image_path).exists()
    print(f"Image exists: {image_exists}")

    if not image_exists:
        print(f"Image not found at path: {image_path}")
        return None

    try:
        # Step 1: Upload image
        print("\nStep 1: Uploading image...")
        img_file = handle_file(image_path)  # Changed from file() to handle_file()
        upload_result = client.predict(
            imge_path=img_file,
            api_name="/upload_image"
        )
        
        # Step 2: Trigger processing
        print("\nStep 2: Triggering processing...")
        process_result = client.predict(
            image_path=img_file,
            files=[],
            slider_value=4,
            api_name="/process_images"
        )
        
        # Extract the paths from process_result
        annotation_path = process_result[1].get('value') if len(process_result) > 1 else None
        annotated_image_path = process_result[2].get('value') if len(process_result) > 2 else None
        
        if annotation_path and annotated_image_path:
            print(f"\nFound result paths:")
            print(f"Annotations: {annotation_path}")
            print(f"Annotated image: {annotated_image_path}")
            
            # Generate output filenames based on input image
            base_name = Path(image_path).stem
            out_annotation = output_path / f"{base_name}_annotations.json"
            out_image = output_path / f"{base_name}_annotated.webp"
            
            # Copy the files
            print("\nSaving results...")
            try:
                # Copy annotation file
                shutil.copy2(annotation_path, out_annotation)
                print(f"Saved annotations to: {out_annotation}")
                
                # Copy annotated image
                shutil.copy2(annotated_image_path, out_image)
                print(f"Saved annotated image to: {out_image}")
                
                return {
                    'annotation_file': str(out_annotation),
                    'annotated_image': str(out_image)
                }
            
            except Exception as e:
                print(f"Error saving files: {str(e)}")
                # If we can't copy, try to read and write the annotation file
                try:
                    with open(annotation_path, 'r') as f:
                        annotations = json.load(f)
                    with open(out_annotation, 'w') as f:
                        json.dump(annotations, f, indent=2)
                    print(f"Saved annotations using direct file read/write")
                    return {'annotation_file': str(out_annotation)}
                except Exception as e2:
                    print(f"Error with direct file handling: {str(e2)}")
                    return None
        else:
            print("No valid result paths found in the process result")
            return None
            
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        return None


In [5]:
img_dir = config['cropped_jpg_location']
success_count = 0
fail_count = 0
failed_files = []

for filename in os.listdir(img_dir):
    print(f"\nProcessing file {success_count + fail_count + 1}: {filename}")
    image_path = os.path.join(img_dir, filename)
    results = process_single_image(image_path)

    if results:
        print("\nSuccess!")
        success_count += 1
        for key, path in results.items():
            print(f"{key}: {path}")
    else:
        print("\nProcessing failed")
        fail_count += 1
        failed_files.append(filename)

print("\nProcessing complete!")
print(f"Successfully processed: {success_count} files")
print(f"Failed to process: {fail_count} files")
if failed_files:
    print("\nFailed files:")
    for f in failed_files:
        print(f"- {f}")


Processing file 1: malawi-cyclone_00000001_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/1b39011abf4677dc67722c74a449a636a8ca8b3b74cb2f38d411b02718135cfb/8bac071a594a510f_annotations.coco.json
Annotated image: /tmp/gradio/62b81ea6d67e254120978751cb5c2bd69ea170b9bc83b2bf7bfcda06c80da80f/image.webp

Saving results...
Saved annotations to: rt_detr_outputs/malawi-cyclone_00000001_pre_disaster_annotations.json
Saved annotated image to: rt_detr_outputs/malawi-cyclone_00000001_pre_disaster_annotated.webp

Success!
annotation_file: rt_detr_outputs/malawi-cyclone_00000001_pre_disaster_annotations.json
annotated_image: rt_detr_outputs/malawi-cyclone_00000001_pre_disaster_annotated.webp

Processing file 2: malawi-cyclone_00000002_post_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annota